<a href="https://colab.research.google.com/github/HayateSato/DS_Practice/blob/main/Bean_imageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import hashlib
from collections import Counter # count the occurrences of each class label in image dataset
from PIL import Image #image processing


In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')

train_path = f'/content/drive/MyDrive/Colab Notebooks/Dataset/Bean_leaf_dataset/train'
test_path = f'/content/drive/MyDrive/Colab Notebooks/Dataset/Bean_leaf_dataset/test'
vali_path = f'/content/drive/MyDrive/Colab Notebooks/Dataset/Bean_leaf_dataset/validation'

working_dir = train_path
os.chdir(working_dir)
os.listdir(working_dir)

Mounted at /content/drive


['angular_leaf_spot', 'healthy', 'bean_rust']

# EDA


In [7]:
# Create datasets for each split
train_ds = tf.keras.utils.image_dataset_from_directory(train_path)
val_ds = tf.keras.utils.image_dataset_from_directory(vali_path)
test_ds = tf.keras.utils.image_dataset_from_directory(test_path)

Found 1034 files belonging to 3 classes.
Found 133 files belonging to 3 classes.
Found 128 files belonging to 3 classes.


In [8]:
class_names = train_ds.class_names
class_names

['angular_leaf_spot', 'bean_rust', 'healthy']

Distribution of Classes.
====

Goal: Check if the dataset is balanced across the three classes: Angular Leaf Spot, Bean Rust, and Healthy.

In [9]:
# import numpy as np
# from collections import Counter

# Get class names from the dataset
class_names = train_ds.class_names

# Iterate through the dataset to collect labels
class_labels = []
for images, labels in train_ds:  # Unpack images and labels from the dataset
    class_labels.extend(labels.numpy())  # Convert labels to NumPy array and extend the list

class_counts = Counter(class_labels)

# Print class distribution
print("Class Distribution:", class_counts)

Class Distribution: Counter({1: 348, 0: 345, 2: 341})


In [10]:
# Get class names from the dataset
class_names = train_ds.class_names

# Iterate through the train_ds to collect labels
class_labels = []
for images, labels in train_ds:  # Unpack images and labels from the dataset
    class_labels.extend(labels.numpy())  # Convert labels to NumPy array and extend the list

class_counts = Counter(class_labels)

# Print class distribution with class names
for label, count in class_counts.items():
    print(f"Class label {label}:  {class_names[label]}")

Class label 1:  bean_rust
Class label 0:  angular_leaf_spot
Class label 2:  healthy
